In [1]:
import io
import os
import json

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/chappers/Documents/DataScience/0_Method_DS/MethodDS-KaggleKat-8b57553f8dbf.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/chappers/Documents/DataScience/0_Method_DS/MethodDS-KaggleKat-8b57553f8dbf.json


In [3]:
# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.join(
    os.path.dirname('__file__'),
    'IMG_2934.jpg')

In [4]:
# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    # print(label.description)
    print(label)

Labels:
mid: "/m/0n5v01m"
description: "bag"
score: 0.8551287651062012
topicality: 0.8551287651062012

mid: "/m/01jwgf"
description: "product"
score: 0.7906313538551331
topicality: 0.7906313538551331

mid: "/m/080hkjn"
description: "handbag"
score: 0.7440301775932312
topicality: 0.7440301775932312

mid: "/m/04t7l"
description: "metal"
score: 0.6029984354972839
topicality: 0.6029984354972839

mid: "/m/03hzcs"
description: "beige"
score: 0.587562084197998
topicality: 0.587562084197998

mid: "/m/0j62f"
description: "rectangle"
score: 0.5806199908256531
topicality: 0.5806199908256531

mid: "/m/0hgrzkp"
description: "shoulder bag"
score: 0.5463389158248901
topicality: 0.5463389158248901



In [5]:
# create str_search to be used in the EBAY API call for KEYWORDS
str_search = str(labels[0].description + " , " +
                 labels[2].description + " , " +
                 # + labels[3].description + " , " +
                 labels[4].description + " , " + 
                 # labels[5].description + " , " +
                 labels[6].description)
str_search

'bag , handbag , beige , shoulder bag'

In [6]:
# call EBAY API and get offers, based on the STR_SEARCH value.
# note, 
from ebaysdk.finding import Connection as Finding
from ebaysdk.exception import ConnectionError

try:
    api = Finding(siteid='EBAY-US', appid="MethodDa-MethodDa-PRD-babd43d69-a4097943", config_file=None)
    response = api.execute('findItemsAdvanced', {'keywords': str_search})
    print(response.dict())
except ConnectionError as e:
    print(e)
    print(e.response.dict())

{'ack': 'Success', 'version': '1.13.0', 'timestamp': '2018-07-09T23:54:18.438Z', 'searchResult': {'item': [{'itemId': '142852393109', 'title': 'White House Black Market Handbag Shoulder Bag Tote Shopper Beige Gold Shiny', 'globalId': 'EBAY-US', 'primaryCategory': {'categoryId': '169291', 'categoryName': "Women's Bags & Handbags"}, 'galleryURL': 'http://thumbs2.ebaystatic.com/m/mVl-H9xKwlsHeu6gKSnc99Q/140.jpg', 'viewItemURL': 'http://www.ebay.com/itm/White-House-Black-Market-Handbag-Shoulder-Bag-Tote-Shopper-Beige-Gold-Shiny-/142852393109', 'paymentMethod': 'PayPal', 'autoPay': 'false', 'postalCode': '66030', 'location': 'Gardner,KS,USA', 'country': 'US', 'shippingInfo': {'shippingType': 'Calculated', 'shipToLocations': 'Worldwide', 'expeditedShipping': 'true', 'oneDayShippingAvailable': 'false', 'handlingTime': '3'}, 'sellingStatus': {'currentPrice': {'_currencyId': 'USD', 'value': '21.98'}, 'convertedCurrentPrice': {'_currencyId': 'USD', 'value': '21.98'}, 'bidCount': '0', 'sellingSta

In [7]:
with open('image_G_output.json', 'w') as outfile:
    json.dump(response.dict(), outfile)

In [8]:
# save the records returned from EBAY API in a formatted json file
def getFileData(fileName):
# Open a file, and read the contents. The with..open operation will auto-close the file as well.
    with open(fileName) as handle:
        data = handle.read()
        return data
def prettyPrint(data, outfile):
# Pretty print and write the file back to the argument 'outfile'
    with open(outfile, "w") as handle:
        handle.write ( json.dumps(json.loads(data), indent=4, sort_keys=True) )
     
outfile = 'tester1.json'
jsondata = getFileData('image_G_output.json')
prettyPrint(jsondata, outfile)
print('Pretty printer complete.')

Pretty printer complete.


In [9]:
# read in the image file (json) that just created
with open('image_G_output.json', 'r') as fp:
    obj = json.load(fp)

In [10]:
# save off the URL's for all offers found
url_list=[]
for row in range(0,len(obj["searchResult"]["item"])):
    url_list.append(obj["searchResult"]["item"][row]["viewItemURL"])
url_list

['http://www.ebay.com/itm/White-House-Black-Market-Handbag-Shoulder-Bag-Tote-Shopper-Beige-Gold-Shiny-/142852393109',
 'http://www.ebay.com/itm/Dooney-Bourke-Handbag-Purse-Beige-Off-White-Shoulder-Bag-/123239936880',
 'http://www.ebay.com/itm/Women-Leather-Handbag-Shoulder-Ladies-Purse-Messenger-Satchel-Crossbody-Tote-Bag-/401408807710?var=671038088831',
 'http://www.ebay.com/itm/Womens-Large-Canvas-Shoulder-Bag-Tote-Handbag-Satchel-Bag-Messenger-Shopping-/163033307654?var=462138387586',
 'http://www.ebay.com/itm/Authentic-New-ysl-Yves-Saint-Laurent-MONOGRAM-beige-handbag-shoulder-bag-/183316126813',
 'http://www.ebay.com/itm/Fashion-Women-Leather-Handbag-Tote-Shoulder-Bag-Purse-Messenger-Crossbody-Bag-/272806676475?var=571946835750',
 'http://www.ebay.com/itm/Beige-Handbag-Shoulder-Bag-Purse-Crossbody-Strap-/113102933053',
 'http://www.ebay.com/itm/Female-Handbag-Messenger-Bag-Shoulder-Bag-Fashion-Bag-Black-Beige-Brown-Green-/362050489696?var=0',
 'http://www.ebay.com/itm/OrYany-Heath